## Временные ряды, часть 2.

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.optimize import minimize

import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('hour_online.csv', index_col=['Time'], parse_dates=['Time'])

In [ ]:
plt.figure(figsize=(12,10))
plt.plot(dataset['Users'])

# Скользящее среднее

### Задание 1. Напишите функцию, вычисляющую скользящее среднее, а именно, по данному ряду series и числу n вычислите среднее значение последних n значений ряда.

In [ ]:
def moving_average(series, n):
    #your code here
    
moving_average(dataset.Users, 24)

In [ ]:
def plotMovingAverage(series, n):

    """
    series - dataframe with timeseries
    n - rolling window size 

    """

    rolling_mean = series.rolling(window=n).mean()

    # При желании, можно строить и доверительные интервалы для сглаженных значений
    #rolling_std =  series.rolling(window=n).std()
    #upper_bond = rolling_mean+1.96*rolling_std
    #lower_bond = rolling_mean-1.96*rolling_std

    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(n))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    #plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
    #plt.plot(lower_bond, "r--")
    plt.plot(dataset[n:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

In [ ]:
plotMovingAverage(dataset, 24) # сглаживаем по дням
plotMovingAverage(dataset, 24*7) # сглаживаем по неделям

# Взвешенное среднее

### Задание 2. Напишите функцию, вычисляющую взвешенное среднее, а именно, вычисляющую сумму последних значений ряда series с весами, лежащими в списке weights.

$$weights[0] \cdot series[-1] + weights[1] \cdot series[-2] + ... + weights[n] \cdot series[-n-1]$$

In [ ]:
def weighted_average(series, weights):
    #your code here
    
weighted_average(dataset.Users, [0.6, 0.2, 0.1, 0.07, 0.03])

# Экспоненциальное сглаживание

### Задание 3. Напишите функцию для экспоненциального сглаживания, которая каждое значение ряда преобразует по формуле:
$$result[n] = alpha \cdot series[n] + (1 - alpha) \cdot result[n-1]$$

In [ ]:
def exponential_smoothing(series, alpha):
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        #your code here
    return result

In [ ]:
with plt.style.context('seaborn-white'):    
    plt.figure(figsize=(20, 8))
    for alpha in [0.3, 0.05]:
        plt.plot(exponential_smoothing(dataset.Users, alpha), label="Alpha {}".format(alpha))
    plt.plot(dataset.Users.values, "c", label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Exponential Smoothing")
    plt.grid(True)

# Двойное экспоненциальное сглаживание.

In [ ]:
def double_exponential_smoothing(series, alpha, beta):
    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # прогнозируем
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
        
#        print('level=',level,' trend=',trend)
        
    return result

In [ ]:
with plt.style.context('seaborn-white'):    
    plt.figure(figsize=(20, 8))
    for alpha in [0.9, 0.02]:
        for beta in [0.9, 0.02]:
            plt.plot(double_exponential_smoothing(dataset.Users, alpha, beta), label="Alpha {}, beta {}".format(alpha, beta))
    plt.plot(dataset.Users.values, label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Double Exponential Smoothing")
    plt.grid(True)

# Тройное экспоненциальное сглаживание (модель Хольта-Винтерса)

In [ ]:
class HoltWinters:

    """
    Модель Хольта-Винтерса с методом Брутлага для детектирования аномалий
    https://fedcsis.org/proceedings/2012/pliks/118.pdf

    # series - исходный временной ряд
    # slen - длина сезона
    # alpha, beta, gamma - коэффициенты модели Хольта-Винтерса
    # n_preds - горизонт предсказаний
    # scaling_factor - задаёт ширину доверительного интервала по Брутлагу (обычно принимает значения от 2 до 3)

    """

    def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=1.96):
        self.series = series
        self.slen = slen
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor

    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i+self.slen] - self.series[i]) / self.slen
        return sum / self.slen  

    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series)/self.slen)
        # вычисляем сезонные средние
        for j in range(n_seasons):
            season_averages.append(sum(self.series[self.slen*j:self.slen*j+self.slen])/float(self.slen))
        # вычисляем начальные значения
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += self.series[self.slen*j+i]-season_averages[j]
            seasonals[i] = sum_of_vals_over_avg/n_seasons
        return seasonals   

    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBond = []
        self.LowerBond = []

        seasonals = self.initial_seasonal_components()

        for i in range(len(self.series)+self.n_preds):
            if i == 0: # инициализируем значения компонент
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i%self.slen])

                self.PredictedDeviation.append(0)

                self.UpperBond.append(self.result[0] + 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])

                self.LowerBond.append(self.result[0] - 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])

                continue
            if i >= len(self.series): # прогнозируем
                m = i - len(self.series) + 1
                self.result.append((smooth + m*trend) + seasonals[i%self.slen])

                # во время прогноза с каждым шагом увеличиваем неопределенность
                self.PredictedDeviation.append(self.PredictedDeviation[-1]*1.01) 

            else:
                val = self.series[i]
                last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                self.result.append(smooth+trend+seasonals[i%self.slen])

                # Отклонение рассчитывается в соответствии с алгоритмом Брутлага
                self.PredictedDeviation.append(self.gamma * np.abs(self.series[i] - self.result[i]) 
                                               + (1-self.gamma)*self.PredictedDeviation[-1])

            self.UpperBond.append(self.result[-1] + 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.LowerBond.append(self.result[-1] - 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i % self.slen])

In [ ]:
with plt.style.context('seaborn-white'):    
    plt.figure(figsize=(20, 8))
    for alpha in [0.9, 0.02]:
        for beta in [0.9, 0.02]:
                for gamma in [0.05]:
                    HW = HoltWinters(dataset.Users, 3, alpha, beta, gamma, 1)
                    HW.triple_exponential_smoothing()
                    plt.plot(HW.result, label="Alpha {}, beta {}, gamma {}".format(alpha, beta, gamma))
    plt.plot(dataset.Users.values, label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Triple Exponential Smoothing")
    plt.grid(True)

# Кросс-валидация

### Задание 4. Разбейте временной ряд series на две части, используя функцию TimeSeriesSplit с количеством фолдов n_splits=3.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

#your code here

In [ ]:
def timeseriesCVscore(x):
    # вектор ошибок
    errors = []

    values = data.values
    alpha, beta, gamma = x

    # задаём число фолдов для кросс-валидации
    tscv = TimeSeriesSplit(n_splits=3) 
    
    # идем по фолдам, на каждом обучаем модель, строим прогноз на отложенной выборке и считаем ошибку
    for train, test in tscv.split(values):

        model = HoltWinters(series=values[train], slen = 24*7, alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
        model.triple_exponential_smoothing()

        predictions = model.result[-len(test):]
        actual = values[test]
        error = mean_squared_error(predictions, actual)
        errors.append(error)

    # Возвращаем средний квадрат ошибки по вектору ошибок 
    return np.mean(np.array(errors))

In [ ]:
%%time
data = dataset.Users[:-500] # отложим часть данных для тестирования

# инициализируем значения параметров
x = [0, 0, 0] 

# Минимизируем функцию потерь с ограничениями на параметры
opt = minimize(timeseriesCVscore, x0=x, method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))

# Из оптимизатора берем оптимальное значение параметров
alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)

In [ ]:
data = dataset.Users
model = HoltWinters(data[:-128], slen = 24*7, alpha = alpha_final, beta = beta_final, gamma = gamma_final, n_preds = 128, scaling_factor = 2.56)
model.triple_exponential_smoothing()

In [ ]:
def plotHoltWinters():
    Anomalies = np.array([np.NaN]*len(data))
    Anomalies[data.values<model.LowerBond] = data.values[data.values<model.LowerBond]
    plt.figure(figsize=(25, 10))
    plt.plot(model.result, label = "Model")
    plt.plot(model.UpperBond, "r--", alpha=0.5, label = "Up/Low confidence")
    plt.plot(model.LowerBond, "r--", alpha=0.5)
    plt.fill_between(x=range(0,len(model.result)), y1=model.UpperBond, y2=model.LowerBond, alpha=0.5, color = "grey")
    plt.plot(data.values, label = "Actual")
    plt.plot(Anomalies, "o", markersize=10, label = "Anomalies")
    plt.axvspan(len(data)-128, len(data), alpha=0.5, color='lightgrey')
    plt.grid(True)
    plt.axis('tight')
    plt.legend(loc="best", fontsize=13);

plotHoltWinters()

# Машинное обучение. 

## Извлечение признаков

In [ ]:
def code_mean(data, cat_feature, real_feature):
    """
    Возвращает словарь, где ключами являются уникальные категории признака cat_feature, 
    а значениями - средние по real_feature
    """
    return dict(data.groupby(cat_feature)[real_feature].mean())

In [ ]:
data.index

In [ ]:
data = pd.DataFrame(dataset)
data.columns = ["y"]

#data.index = data.index.to_datetime()
data["hour"] = data.index.hour

### Задание 5. Создайте столбец weekda в таблице data, куда запишите день недели, взятый из даты, находящейся в data.index. После этого создайте столбец is_weekend, в который запишите 1, если день недели выходной, и 0 - иначе. 

In [ ]:
#your code here

data.head()

In [ ]:
code_mean(data, 'weekday', "y")

Очень хорошими признаками для решения задач, связанных с временными рядами с помощью машинного обучения, являются лаги. Лаг - это ряд, сдвинутый на несколько моментов времени назад.

In [ ]:
print(data.y.shift(3))

### Задание 6. Выведите на экран (с помощью цикла) последние 10 значений первых 10 лагов ряда: сначала 10 значений самого ряда, затем 10 значений ряда, свдинутого на 1, затем 10 значений ряда, сдвинутого на 2 и т.д.

In [ ]:
#your code here

Напишем функцию для извлечения всех рассмотренных выше признаков (средние значения, признаки по дате, лаговые признаки).

In [ ]:
def prepareData(data, lag_start=5, lag_end=20, test_size=0.15):

    data = pd.DataFrame(data.copy())
    data.columns = ["y"]

    # считаем индекс в датафрейме, после которого начинается тестовый отрезок
    test_index = int(len(data)*(1-test_size))

    # добавляем лаги исходного ряда в качестве признаков
    for i in range(lag_start, lag_end):
        data["lag_{}".format(i)] = data.y.shift(i)

#    data.index = data.index.to_datetime()
    data["hour"] = data.index.hour
    data["weekday"] = data.index.weekday
    data['is_weekend'] = data.weekday.isin([5,6])*1

    # считаем средние только по тренировочной части, чтобы избежать лика

    data['weekday_average'] = list(map(code_mean(data[:test_index], 'weekday', "y").get, data.weekday))
    data["hour_average"] = list(map(code_mean(data[:test_index], 'hour', "y").get, data.hour))
    # выкидываем закодированные средними признаки 
    data.drop(["hour", "weekday"], axis=1, inplace=True)

    data = data.dropna()
    data = data.reset_index(drop=True)

    # разбиваем весь датасет на тренировочную и тестовую выборку
    X_train = data.loc[:test_index].drop(["y"], axis=1)
    y_train = data.loc[:test_index]["y"]
    X_test = data.loc[test_index:].drop(["y"], axis=1)
    y_test = data.loc[test_index:]["y"]

    return X_train, X_test, y_train, y_test

## Линейная регрессия

In [ ]:
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = prepareData(dataset.Users, test_size=0.3, lag_start=1, lag_end=48)
X_train.head()

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X_test)
plt.figure(figsize=(15, 7))
plt.plot(prediction, "r", label="prediction")
plt.plot(y_test.values, label="actual")
plt.legend(loc="best")
plt.title("Linear regression\n Mean absolute error {} users".format(round(mean_absolute_error(prediction, y_test))))
plt.grid(True);

In [ ]:
def performTimeSeriesCV(X_train, y_train, number_folds, model, metrics):
    print('Size train set: {}'.format(X_train.shape))

    k = int(np.floor(float(X_train.shape[0]) / number_folds))
    print('Size of each fold: {}'.format(k))

    errors = np.zeros(number_folds-1)

    # loop from the first 2 folds to the total number of folds    
    for i in range(2, number_folds + 1):
        print('')
        split = float(i-1)/i
        print('Splitting the first ' + str(i) + ' chunks at ' + str(i-1) + '/' + str(i) )

        X = X_train[:(k*i)]
        y = y_train[:(k*i)]
        print('Size of train + test: {}'.format(X.shape)) # the size of the dataframe is going to be k*i

        index = int(np.floor(X.shape[0] * split))

        # folds used to train the model        
        X_trainFolds = X[:index]        
        y_trainFolds = y[:index]

        # fold used to test the model
        X_testFold = X[(index + 1):]
        y_testFold = y[(index + 1):]

        model.fit(X_trainFolds, y_trainFolds)
        errors[i-2] = metrics(model.predict(X_testFold), y_testFold)

    # the function returns the mean of the errors on the n-1 folds    
    return errors.mean()

In [ ]:
%%time
performTimeSeriesCV(X_train, y_train, 5, lr, mean_absolute_error)

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def RandomForest_forecast(data, lag_start=5, lag_end=20, test_size=0.3, scale=1.96):

    X_train, X_test, y_train, y_test = prepareData(dataset.Users, lag_start, lag_end, test_size)

    RF = RandomForestRegressor(n_estimators=100)
    RF.fit(X_train, y_train)
    
    # посмотрим, как модель вела себя на тренировочном отрезке ряда
    prediction_train = RF.predict(X_train)

    plt.figure(figsize=(15, 5))
    plt.plot(prediction_train)
    plt.plot(y_train)
    plt.axis('tight')
    plt.grid(True)

    # и на тестовом
    prediction_test = RF.predict(X_test)
    
    plt.figure(figsize=(15, 5))
    plt.plot(prediction_test, label="prediction")
    plt.plot(list(y_test), label="y_test")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("RandomForest Mean absolute error {} users".format(round(mean_absolute_error(prediction_test, y_test))))
    plt.grid(True)
    plt.legend()

In [ ]:
RandomForest_forecast(dataset, test_size=0.3, lag_start=1, lag_end=24)

# Facebook Prophet

In [ ]:
from fbprophet import Prophet

In [ ]:
dataset = pd.read_csv('hour_online.csv', index_col=['Time'], parse_dates=['Time'])
d = dataset.reset_index()

In [ ]:
d.head()

In [ ]:
d.rename(columns = {'Time':'ds', 'Users':'y'}, inplace = True)
print(d.head())

train_dataset = d.iloc[:-800]
test_dataset = d.iloc[-800:]

In [ ]:
d.dtypes

In [ ]:
m = Prophet() #daily_seasonality, weekly_seasonality
m.fit(d_train)

In [ ]:
future = m.make_future_dataframe(freq='H',periods=300) #freq='H' означает, что измерения проводятся каждый час
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
prediction = forecast['yhat'][-800:]
actual = d_test['y']
mean_absolute_error(prediction,actual)

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
%pylab inline
plot(forecast['yhat'][-500:])

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
metric_df = forecast.set_index('ds')[['yhat']].join(d.set_index('ds').y).reset_index()
metric_df.tail()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(metric_df.yhat, label="prediction")
plt.plot(list(metric_df.y), label="y_test")
plt.legend(loc="best")
plt.axis('tight')
plt.title("FBProphet Mean absolute error {} users".format(round(mean_absolute_error(metric_df.yhat, metric_df.y))))
plt.grid(True)
plt.legend()

## Данные по временным рядам 

https://data.world/datasets/time-series

https://archive.ics.uci.edu/ml/datasets.php?format=&task=&att=&area=&numAtt=10to100&numIns=&type=ts&sort=nameUp&view=table

# Домашнее задание

Скачайте с одного из предложенных сайтов (или с любого другого) наиболее понравившийся вам датасет и далее работайте с ним.

Разбейте данные на train и test (test - последние 15%-30% данных в зависимости от размера датасета).

1) Постройте адаптивные модели двойного экспоненциального сглаживания и тройного экспоненциального сглаживания. 
Сделайте предсказания и измерьте ошибки mse и mae на train и test.

2*) Исследуйте данные на стационарность. Проверьте наличие сезонных эффектов, зависимостей от предыдущих значений ряда, сбалансируйте дисперсию с помощью преобразования Бокса-Кокса. Приведите ряд к стационарному и примените к полученному стационарному ряду модель ARMA. Подберите параметры p и q для ARMA.

3) Постройте предсказание с помощью модели ARIMA (или SARIMA, если вы подозреваете наличие сезонных эффектов в данных). Измерьте ошибки предсказаний на train и test.

4) Сделайте предсказание с помощью fbprophet и измерьте mse и mae предсказания на train и на test.

5) Поработайте над созданием новых признаков (лаги, аггрегированные признаки и т.д.) и примените для предсказания линейную регрессию и какой-нибудь нелинейный алгоритм.

6) Сравните результаты всех полученных алгоритмов. Какой алгоритм дал наилучшее качество? Какой алгоритм имеет самое маленькое время настройки параметров при хорошем качестве?

7*) Попробуйте скомбинировать наилучшие модели. Улучшилось ли качество предсказания?

Задания 2 и 7 опциональные.